In [2]:
#!pip install keras-tuner

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [7]:
fashion_mnist = keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [10]:
#converting all pixels in the images to values between 0 to 1
train_images = train_images/255.0
test_images = test_images/255.0

In [12]:
train_images[0].shape

(28, 28)

In [15]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [33]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model
    


In [31]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [34]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

In [35]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 07m 13s]
val_accuracy: 0.9120000004768372

Best val_accuracy So Far: 0.9120000004768372
Total elapsed time: 00h 38m 51s
INFO:tensorflow:Oracle triggered exit


In [38]:
model=tuner_search.get_best_models(num_models=1)[0] #selecting best model

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,140,298
Trainable params: 3,140,298
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 146s 86ms/step - loss: 0.1119 - accuracy: 0.9589 - val_loss: 0.2743 - val_accuracy: 0.9118
Epoch 5/10
1688/1688 [==============================] - 133s 79ms/step - loss: 0.0740 - accuracy: 0.9726 - val_loss: 0.3367 - val_accuracy: 0.9130
Epoch 6/10
1688/1688 [==============================] - 135s 80ms/step - loss: 0.0506 - accuracy: 0.9817 - val_loss: 0.3728 - val_accuracy: 0.9140
Epoch 7/10
1688/1688 [==============================] - 135s 80ms/step - loss: 0.0342 - accuracy: 0.9879 - val_loss: 0.4006 - val_accuracy: 0.9117
Epoch 8/10
1688/1688 [==============================] - 134s 79ms/step - loss: 0.0276 - accuracy: 0.9900 - val_loss: 0.4721 - val_accuracy: 0.9160
Epoch 9/10
1688/1688 [==============================] - 134s 80ms/step - loss: 0.0210 - accuracy: 0.9930 - val_loss: 0.5293 - val_accuracy: 0.9155
Epoch 10/10
1688/1688 [==============================] - 143s 85ms/step - loss: 0.0210 - accuracy: 0.9928 - val_loss: 